<a href="https://colab.research.google.com/github/wcrowley342/bookanalysis/blob/main/Book_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro & Read in Data

In [3]:

# Some basic, common libraries used:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
from matplotlib import dates as mpl_dates
from matplotlib import rcParams
import seaborn as sns
import plotly as py
import plotly.express as px

#Read in data into "titles" data frame

titles_df = pd.read_csv('https://raw.githubusercontent.com/wcrowley342/bookanalysis/main/titles.csv')

In [4]:
titles_df.head()

,id,title,author,year,total_weeks,first_week,debut_rank,best_rank
0,0,"""H"" IS FOR HOMICIDE",Sue Grafton,1991,15,1991-05-05,15,2
1,1,"""I"" IS FOR INNOCENT",Sue Grafton,1992,11,1992-04-26,2,2
2,10,''G'' IS FOR GUMSHOE,Sue Grafton,1990,6,1990-05-06,8,8
3,100,A DOG'S JOURNEY,W. Bruce Cameron,2012,1,2012-05-27,14,14
4,1000,CHANGING FACES,Kimberla Lawson Roby,2006,1,2006-02-19,14,14


In [6]:
titles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7431 entries, 0 to 7430
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0    id          7431 non-null   int64 
 1   title        7431 non-null   object
 2   author       7427 non-null   object
 3   year         7431 non-null   int64 
 4   total_weeks  7431 non-null   int64 
 5   first_week   7431 non-null   object
 6   debut_rank   7431 non-null   int64 
 7   best_rank    7431 non-null   int64 
dtypes: int64(5), object(3)
memory usage: 464.6+ KB


# CLEAN UP DATA

# LONGEST TENURED AUTHOR

# AUTHOR WITH MOST WEEKS

# AUTHOR WITH MOST TITLES/MOST PROLIFIC